## Importing libraries

In [0]:
import json
import urllib
import requests
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup

## extracting urls and their text from warc.gz files from common crawl indexes

In [0]:
def write_data(rrr, filename):
    soup = BeautifulSoup(rrr, 'html.parser')
    f = open(filename, "a+")
    for xx in soup.find_all('p'):
        f.write(xx.text.encode("utf-8"))
        f.write("\n")
    f.close()

def extract_data(uri, filename):
    c = 0
    response = requests.get(uri) ##fetching the json file from common crawl for each subtopic
    s = ''
    fList = []
    for z in response.text: #isolating each record in the json file
        if z=="\n":
            data = json.loads(s) #loading and extracting the offset, length and the warc file name from the record
            fList.append([data["filename"].encode("utf-8"), data["offset"], data["length"]])
            s = ''
        else:
            s += z
    links = []
    for x in fList: #iterating through all the warc files obtained from json
        offset = int(x[1]) #calculating the total length of the actual data required
        end = offset + int(x[2]) - 1
        try:
            resp = requests.get("https://commoncrawl.s3.amazonaws.com/"+x[0], headers={'Range':'bytes={}-{}'.format(offset, end)},stream=True)
        except: #hitting every warc files in s3 bucket
            print("bad connection")
            continue
        for record in ArchiveIterator(resp.raw, arc2warc=True): #unzipping warc files 
            if record.rec_type == 'response':
            uri = record.rec_headers.get_header('WARC-Target-URI')  
            sepl = uri.find(".com") #checking if links are not duplicate and redundant
            uri = uri[sepl+4:] 
            if uri in links:
                continue
            links.append(uri)
            if 'story' in uri or 'news' in uri: #making sure the URLs are articles or news
                print(uri)
                write_data(record.content_stream().read(), filename) #writing those html data to a file
                c += 1
            else:
                continue
            if c > 200: #collecting at most 200 articles/links for each sub topic
                break
        if c > 200:
            break


In [7]:
extract_data("http://index.commoncrawl.org/CC-MAIN-2019-13-index?url=cbssports.com%2F*&output=json", "sports_raw.txt")
extract_data("http://index.commoncrawl.org/CC-MAIN-2019-13-index?url=espn.com%2Fmlb%2F*&output=json", "baseball_raw.txt")
extract_data("https://index.commoncrawl.org/CC-MAIN-2019-13-index?url=http%3A%2F%2Fwww.espn.com%2Fgolf%2F*&output=json", "golf_raw.txt")
extract_data("https://index.commoncrawl.org/CC-MAIN-2019-13-index?url=http%3A%2F%2Fwww.espn.com%2Fnba%2F*&output=json", "basketball_raw.txt")
extract_data("https://index.commoncrawl.org/CC-MAIN-2019-13-index?url=http%3A%2F%2Fwww.espn.com%2Fsoccer%2F*&output=json", "soccer_raw.txt")
extract_data("https://index.commoncrawl.org/CC-MAIN-2019-13-index?url=http%3A%2F%2Fwww.espn.com%2Ftennis%2F*&output=json", "tennis_raw.txt")


/aaf/news/2019-aaf-standings-lets-take-a-look-at-how-all-the-teams-stack-up-in-both-divisions/
/aaf/news/2019-aaf-win-totals-from-the-hotshots-to-the-iron-heres-where-one-sportsbook-puts-the-lines/
/aaf/news/aaf-2019-a-j-tarpley-who-went-from-the-nfl-to-wall-street-and-back-isnt-done-with-football/
/aaf/news/aaf-2019-five-things-to-love-about-the-new-alliance-of-american-football-and-five-things-not-to-love/
/aaf/news/aaf-2019-from-the-nfl-to-wall-street-and-back-a-j-tarpley-isnt-done-with-football/
/aaf/news/aaf-2019-schedule-and-scores-dates-times-tv-channel-streaming-every-game-for-the-entire-season/
/aaf/news/aaf-2019-scores-and-schedule-times-dates-streaming-tv-channel-every-game-for-the-entire-season/
/aaf/news/aaf-2019-standings-lets-take-a-look-at-how-the-teams-stack-up-in-both-divisions/
/aaf/news/aaf-2019-steve-spurrier-is-back-in-his-comfort-zone-coaching-the-orlando-apollos-in-the-alliance-of-american-football/
/aaf/news/aaf-2019-will-johnny-manziel-sign-will-steve-spurrier